## Exploring Gradient Descent, Polynomial Regression and Regularization Techniques using IRIS Data Set

IRIS is a simple dataset that comprises the `sepal` and `petal` dimensions of 3 types of IRIS flowers. 

I'll be using this dataset to test out a couple of algorithms for **classification** i.e. `Logistic Regression` and `Softmax Regression`.

I'll be exploring 2 versions of **Gradient Descent** i.e. `Batch` and `Stochastic`

I'll be also be experimenting with a few techniques for **regularization** i.e. `Ridge`and `Lasso` techniques. 

Additionally, I have also implemented a version of regularization that stops the training process as soon as the generalization error starts increasing i.e. `Early Stopping`. As I dicovered at my end, as others have, that it makes `Stochastic Gradient Descent` converge much **better** 

<br>

Given the large set of things on agenda, the table of contents below should be helpful.



**Table of Contents**
1. **Exploring IRIS Data**
2. **Preparing the Data for Logistic *(k=2)* and Multiclass *(k=3)* usecases**
3. **(2 Class) Logistic Regression with two varients of Gradient Descent - Batch and Stochastic**
4. **(2 class) SGD with *quasi* regulartization via Early Stopping - does it give better results than *vanilla* SGD?**
5. **(2 class) Exploring Polynomial Logistic Regression**
6. **(2 class) Stochastic Gradient Descent with `Ridge` and `Lasso` regularizations**


***


## Exploring Data



In [5]:
from sklearn import datasets
iris = datasets.load_iris()

list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']

In [7]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [18]:
X = iris["data"]
type(X)

print(X.shape,"\n")

print(X[0:5,:])

(150, 4) 

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [31]:
y = iris["target"]
type(y)

print(y.shape,"\n")

print(y[0:5])

y = y.reshape((150,1))

print(y.shape)

(150,) 

[0 0 0 0 0]
(150, 1)


## Preparing Data for k=2 and k = 3 use cases

In [36]:
import numpy as np
iris = np.hstack([X,y])

print(iris[48:52,:])

# splitting data into training and validation sets
from sklearn.model_selection import train_test_split

train, val = train_test_split(iris,test_size = 0.2, random_state = 42)

print("training set: ",train.shape)
print('\n')
print("test set: ",val.shape)

[[5.3 3.7 1.5 0.2 0. ]
 [5.  3.3 1.4 0.2 0. ]
 [7.  3.2 4.7 1.4 1. ]
 [6.4 3.2 4.5 1.5 1. ]]
training set:  (120, 5)


test set:  (30, 5)


In [44]:
# quickly checking of the training set is representative across the three class
import pandas as pd
train_df = pd.DataFrame(train, columns = ['a','b','c','d','class'])

pd.pivot_table(train_df, index = ['class'],aggfunc=len, margins = True )  #Looks good, lets move on.

,a,b,c,d
class,,,,
0.0,40.0,40.0,40.0,40.0
1.0,41.0,41.0,41.0,41.0
2.0,39.0,39.0,39.0,39.0
All,120.0,120.0,120.0,120.0


In [60]:
X_train = train[:,0:4]
y_train_3cls = np.array(train[:,4],dtype = int)

X_val = val[:,0:4]
y_val_3cls = np.array(val[:,4],dtype = int)


print("sample data: ",y_train_3cls[48:52])
print("sample data: ",X_train[48:52,:])


print("X_train Shape: ",X_train.shape)
print("Y_train Shape: ",y_train_3cls.shape)


y_train_2cls = (y_train_3cls == 2)
y_val_2cls = (y_val_3cls == 2)


sample data:  [0 1 2 0]
sample data:  [[5.4 3.9 1.7 0.4]
 [5.  2.3 3.3 1. ]
 [6.4 2.7 5.3 1.9]
 [5.  3.3 1.4 0.2]]
X_train Shape:  (120, 4)
Y_train Shape:  (120,)


In [78]:
sample = np.ones((5,1),dtype = int)
sample

array([[1],
       [1],
       [1],
       [1],
       [1]])

#### Quick Summary

We have created the test and validation datsets. There are 2 varients of the labels, one for the 2class implementation of IRIS and the other for the 3class implementation i.e. `X_train`, `X_val`, `y_train_2cls`, `y_train_3cls`, `y_val_2cls`, `y_val_3cls` 

***

## (2 Class) Logistic Regression with two varients of Gradient Descent - Batch and Stochastic

<br>

#### Creating Pipeline for Automating Data Prep

In [81]:

# a simple function to add a column of ones (this is needed to implement gradient descent)
def one_adder(an_array):
    m = an_array.shape[0]
    pad_one = np.ones((m,1),dtype = int)
    an_array = np.hstack([pad_one,an_array])
    return an_array

#pipeline to automate data prep
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer


dataprep_2cls = Pipeline([
        ("std_scaler", StandardScaler()),
        ("one_adder", FunctionTransformer(one_adder)),
    ])



#### Implementing and Testing Pipeline

In [82]:
print(X_train.shape)
print(X_train[0:5,:])

print("\n")

X_train_t = dataprep_2cls.fit_transform(X_train) 

print(X_train_t.shape)
print(X_train_t[0:5,:])


(120, 4)
[[4.6 3.6 1.  0.2]
 [5.7 4.4 1.5 0.4]
 [6.7 3.1 4.4 1.4]
 [4.8 3.4 1.6 0.2]
 [4.4 3.2 1.3 0.2]]


(120, 5)
[[ 1.         -1.47393679  1.20365799 -1.56253475 -1.31260282]
 [ 1.         -0.13307079  2.99237573 -1.27600637 -1.04563275]
 [ 1.          1.08589829  0.08570939  0.38585821  0.28921757]
 [ 1.         -1.23014297  0.75647855 -1.2187007  -1.31260282]
 [ 1.         -1.7177306   0.30929911 -1.39061772 -1.31260282]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


### Implemention of Batch Gradient Descent
Measure `classification accuracy` on training data

In [327]:
m = y_train_2cls.shape[0]
y_train_2cls = y_train_2cls.reshape((m,1))
y_val_2cls = y_val_2cls.reshape((30,1))


In [284]:
np.random.seed(42)
theta_bgd = np.random.randn(5,1)
eta = 0.001
n_iter = 10000
m = X_train_t.shape[0]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))



for i in range(n_iter):

    pred = sigmoid(X_train_t @ theta_bgd)
    #print(pred.shape)

    error = pred - y_train_2cls
    #print(error.shape)

    grad = (1/m) * (X_train_t.T @ (error) )
    #print(grad.shape)

    theta_bgd = theta_bgd - (eta * grad)

#print("theta.shape",theta_bgd.shape)
print("theta",theta_bgd)
    
    

theta [[-0.86331203]
 [ 0.00408211]
 [ 0.50284986]
 [ 1.94127509]
 [ 0.41202161]]


In [285]:
#training set predictions
Y_train_2cls_probab = sigmoid(X_train_t @ theta_bgd)
Y_train_2cls = Y_train_2cls_probab > 0.5
    
    
    
#training set accuracy
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train_2cls,Y_train_2cls)

classification_accuracy = (cm[0,0] + cm [1,1]) /np.sum(cm)
print(classification_accuracy)



0.875


**Summary**
The Batch Gradient Descent results were evaualted with a learning rate of .001

As expected, the classification accuracy got better as the number of iterations increased. I have summarized the details below:
* 70% at 10 Iterations
* 70% at 100 Iterations
* 73% at 500 Iterations
* 76% at 1000 Iterations
* 81% at 5000 Iterations
* 87% at 10,000 Iterations
* 96% at 50,000 Iterations
* 97.5% at 100,000 Iterations
* 98.3% at 1,000,000 Iterations

***

Next up we'll we exploring **Stochastic Gradient Descent** with the similar learning rate with diffent `epoch` (number of iterations) combinations.

### Implementation of Stochastic Gradient Descent
Measure `classification accuracy` on training data

In [243]:
np.random.seed(42)
theta_sgd = np.random.randn(5,1) 

n_epoches = 100000
iter_count = 0

eta = 0.001

for epoch in range(n_epoches):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_train_t[random_index:random_index+1]
        yi = y_train_2cls[random_index:random_index+1]
        grad = 2 * xi.T @ (xi @ theta_sgd  - yi)
        theta_sgd = theta_sgd - eta * grad
        iter_count+=1
        

print("sgd: ",iter_count ," iterations (epoch*m)",theta_sgd)

sgd:  12000000  iterations (epoch*m) [[ 0.32377531]
 [-0.04799941]
 [ 0.09378805]
 [ 0.05801697]
 [ 0.37015748]]


In [244]:
#training set predictions
Y_train_2cls_probab = sigmoid(X_train_t @ theta_sgd)
Y_train_2cls = Y_train_2cls_probab > 0.5
    
    
    
#training set accuracy
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train_2cls,Y_train_2cls)

classification_accuracy = (cm[0,0] + cm [1,1]) /np.sum(cm)
print(classification_accuracy)



0.5916666666666667


**Summary**
Unlike `Batch Gradient Descent (BGD)`, the calculations for `Shochastic Gradient Descent (SGD)` are **not vectorized**. The compututaional costs for this implementation are to the order or `O(n^2)` given the two `for` loops that have been used. Also, `SGD` doesn't converage in a *linear fashion* die to the inherent *randomness*

**An important point to note that SGD is an 'out of core' algorithm and hence suiable for large datasets that cannot be loaded in memory, all at once** 

Here are the accuracy results as a function of the number of `epoch` and its clearly not performing as well as `BGD`. The `learning rate` has been set at 0.001 as was done earlier
* 58.3% at 10 epoch
* 60% at 500 epoch
* 59% at 1000 epoch
* 60% at 10,000 epoch
* 60% at 50,000 epoch

***
**At this point its worthwhile exploring if `SGD` performs any better with `Early Stopping` implemented.**

## (2 class) SGD with *quasi* regulartization via Early Stopping - does it give better results than *vanilla* SGD?

In [259]:
np.random.seed(42)
theta_sgd_es = np.random.randn(5,1)

classification_accuracy_benchmark = 0
theta_sgd_es_becnhmark = np.random.randn(5,1)
best_epoch = None

n_epoches = 1000
iter_count = 0

eta = 0.001

for epoch in range(n_epoches):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_train_t[random_index:random_index+1]
        yi = y_train_2cls[random_index:random_index+1]
        grad = 2 * xi.T @ (xi @ theta_sgd  - yi)
        theta_sgd = theta_sgd - eta * grad
        iter_count+=1
    
    Y_train_2cls_probab = sigmoid(X_train_t @ theta_sgd)
    Y_train_2cls = Y_train_2cls_probab > 0.5
    
    
    
    #training set accuracy
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_train_2cls,Y_train_2cls)

    classification_accuracy = (cm[0,0] + cm [1,1]) /np.sum(cm)
    if classification_accuracy > classification_accuracy_benchmark:
        classification_accuracy_benchmark = classification_accuracy
        theta_sgd_es_benchmark = theta_sgd_es
        best_epoch= epoch

        

print("sgd + Early Stoppage: ",best_epoch ," epoches",theta_sgd_es_benchmark)
print("sgd + Early Stoppage + classification accuracy ",classification_accuracy_benchmark)


sgd + Early Stoppage:  743  epoches [[ 0.49671415]
 [-0.1382643 ]
 [ 0.64768854]
 [ 1.52302986]
 [-0.23415337]]
sgd + Early Stoppage + classification accuracy  0.6166666666666667


**Summary and Conclusions**
`SGD + Early Stoppage (SGD + ES)` **always** performs better than `SGD`. This is quite **significant**

Here are the comparisons:
* Epoch = 10   | SGD = 58% | SGD + ES = 60%
* Epoch = 100  | SGD = 58% | SGD + ES = 61%
* Epoch = 1000 | SGD = 60% | SGD + ES = 62%
* Epoch = 10000| SGD = 60% | SGD + ES = 63%

***

## (2 class) Exploring Polynomial Regression



#### Data Preperation

In [291]:
from sklearn.preprocessing import PolynomialFeatures 
poly_features = PolynomialFeatures(degree = 2, include_bias = False)

X_train_poly = poly_features.fit_transform(X_train)

X_train_poly_t = dataprep_2cls.fit_transform(X_train_poly)

print(X_train_poly_t.shape)

(120, 15)


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


#### Batch Gradient Descent with Polynomial Features

In [311]:
np.random.seed(42)
theta_poly_bgd = np.random.randn(15,1)
eta = 0.001
n_iter = 2000000
m = X_train_poly_t.shape[0]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))



for i in range(n_iter):

    pred = sigmoid(X_train_poly_t @ theta_poly_bgd)
    #print(pred.shape)

    error = pred - y_train_2cls
    #print(error.shape)

    grad = (1/m) * (X_train_poly_t.T @ (error) )
    #print(grad.shape)

    theta_poly_bgd = theta_poly_bgd - (eta * grad)

#print("theta.shape",theta_bgd.shape)
#print("theta",theta_poly_bgd)
    
#training set predictions
Y_train_2cls_probab = sigmoid(X_train_poly_t @ theta_poly_bgd)
Y_train_2cls = Y_train_2cls_probab > 0.5
    
    
    
#training set accuracy
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train_2cls,Y_train_2cls)

classification_accuracy = (cm[0,0] + cm [1,1]) /np.sum(cm)
print(classification_accuracy)


0.9833333333333333


I had expected `BGD + Polynomial Features` to perform much better than `BGD` on the training set.

However, that was not the case, maybe its because the dataset is relatively simple and BGD does a great job in its native/vanilla form.

Model accuracy % by iterations:
* 10 iterations | Accuracy: 28%
* 100 iterations| Accuracy: 23%
* 1000 iterations| Accuracy: 61%
* 10,000 iterations| Accuracy: 82%
* 100,000 iterations| Accuracy: 97%
* 1,000,000 iterations| Accuracy: 98.3%

***

## (2 class) Stochastic Gradient Descent with `Ridge` and `Lasso` Regularization Techniques

The following are the goals:
1. Comparing the performance of unregularized logistic model with `Ridge` Regularized and `Lasso` Regularized versions.


#### Data Preperation

In [314]:
std_scaler = StandardScaler()

X_train_poly = poly_features.fit_transform(X_train)
X_val_poly = poly_features.fit_transform(X_val)

X_train_prep = std_scaler.fit_transform(X_train_poly)
X_val_prep = std_scaler.fit_transform(X_val_poly)

print(X_train_prep.shape)
print(X_val_prep.shape)

(120, 14)
(30, 14)


#### Training SGD Classifiers

In [318]:
from sklearn.linear_model import SGDClassifier


sgd_model = SGDClassifier(random_state = 42, loss = 'log', penalty = None)
# loss = log denotes a logictic model
# penalty = None, inplies that no regularization has been used

sgd_ridge_model = SGDClassifier(random_state = 42, loss = 'log', penalty = 'l2')
sgd_lasso_model = SGDClassifier(random_state = 42, loss = 'log', penalty = 'l1')

#### Fitting the model and Generating Predictions

In [328]:
sgd_model.fit(X_train_prep,y_train_2cls)
sgd_ridge_model.fit(X_train_prep,y_train_2cls)
sgd_lasso_model.fit(X_train_prep,y_train_2cls)

Y_train_sgd = sgd_model.predict(X_train_prep)
Y_train_ridge_sgd = sgd_ridge_model.predict(X_train_prep)
Y_train_lasso_sgd = sgd_lasso_model.predict(X_train_prep)

Y_val_sgd = sgd_model.predict(X_val_prep)
Y_val_ridge_sgd = sgd_ridge_model.predict(X_val_prep)
Y_val_lasso_sgd = sgd_lasso_model.predict(X_val_prep)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will

#### Evaluating Performance

In [330]:
from sklearn.metrics import precision_score, recall_score, f1_score

def model_performance_scores(train,predicted):
    precision_kpi = precision_score(train,predicted)
    recall_kpi = recall_score(train, predicted)
    f1_kpi = f1_score(train,predicted)
    
    print("model performance scores: ","\n")
    print("Precision = ", precision_kpi)
    print("Recall = ", recall_kpi)
    print("F1 = ", f1_kpi)
    
    print("\n")

print("training + SGD: ")
print(model_performance_scores(y_train_2cls,Y_train_sgd) )
print("============================")
print("training + SGD + Ridge: ")
print(model_performance_scores(y_train_2cls,Y_train_ridge_sgd) )
print("============================")
print("training + SGD + Lasso: ")
print(model_performance_scores(y_train_2cls,Y_train_lasso_sgd) )

print("============================")


training + SGD: 
model performance scores:  

Precision =  0.9069767441860465
Recall =  1.0
F1 =  0.951219512195122


None
training + SGD + Ridge: 
model performance scores:  

Precision =  0.972972972972973
Recall =  0.9230769230769231
F1 =  0.9473684210526315


None
training + SGD + Lasso: 
model performance scores:  

Precision =  0.9743589743589743
Recall =  0.9743589743589743
F1 =  0.9743589743589743


None


**Summary**
Both the `ridge` and `lasso` regularized models perform better than the non-regularized version of the algorithm 